In [1]:
import pandas as pd
import numpy as np
from datetime import date

In [2]:
MODE = 'test' #'train' or 'test'

# Data

In [3]:
data = pd.read_csv('data/'+ MODE +'.csv')
data.head()

,client_id
0,a9a604ed6e
1,ebd7360016
2,908cd9b8e8
3,dceb8ce861
4,f4f0ac6b06


In [4]:
purchase = pd.read_csv('data/'+ MODE +'_purch_hist.csv')
purchase.head()

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,a375a035b9,2.0,50.0,NaN
1,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,6f9e02ee36,1.0,90.0,NaN
2,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,42e1eb8d23,1.0,20.0,NaN
3,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,5c6de3010f,2.0,85.0,NaN
4,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,343e841aaa,1.0,58.0,NaN


In [5]:
clients = pd.read_csv('data/clients.csv').drop(columns=['client_id.1'])
clients.head()

,client_id,first_issue_date,first_redeem_date,age,gender
0,000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U
1,000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F
2,00010925a5,2018-07-24 16:21:29,2018-09-14 16:12:49,83,U
3,0001f552b0,2017-06-30 19:20:38,2018-08-28 12:59:45,33,F
4,00020e7b18,2017-11-27 11:41:45,2018-01-10 17:50:05,73,U


In [6]:
products = pd.read_csv('data/products.csv')
products.head()

,product_id,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol
0,0003020d3c,c3d3a8e8c6,c2a3ea8d5e,b7cda0ec0c,6376f2a852,123.0,394a54a7c1,9eaff48661,0.400,0,0
1,0003870676,e344ab2e71,52f13dac0c,d3cfe81323,6dc544533f,105.0,acd3dd483f,10486c3cf0,0.680,0,0
2,0003ceaf69,c3d3a8e8c6,f2333c90fb,419bc5b424,f6148afbc0,271.0,f597581079,764e660dda,0.500,0,0
3,000701e093,ec62ce61e3,4202626fcb,88a515c084,48cf3d488f,172.0,54a90fe769,03c2d70bad,0.112,0,0
4,0007149564,e344ab2e71,52f13dac0c,d3cfe81323,6dc544533f,105.0,63417fe1f3,f329130198,0.600,0,0


# Dataset

## Age

In [7]:
data = data.merge(clients[['client_id', 'age']], on='client_id', how='left')
# data.loc[data.age > 90, 'age'] = [np.random.randint(10, 90) for _ in range((data.age > 90).sum())]
# data.loc[data.age < 10, 'age'] = [np.random.randint(10, 90) for _ in range((data.age < 10).sum())]
data.head()

,client_id,age
0,a9a604ed6e,36
1,ebd7360016,63
2,908cd9b8e8,49
3,dceb8ce861,46
4,f4f0ac6b06,119


## Gender

In [8]:
data = data.merge(clients[['client_id', 'gender']], on='client_id', how='left')
# data['gender'] = data['gender'].apply(lambda g: ['U', 'M', 'F'].index(g) - 1)
# p = np.array([(data.gender == 0).sum(), (data.gender == 1).sum()]) / (data.gender != -1).sum()
# data.loc[data.gender == -1, 'gender'] = [np.random.choice([0, 1], p=p) for _ in range((data.gender == -1).sum())]
data.head()

,client_id,age,gender
0,a9a604ed6e,36,F
1,ebd7360016,63,F
2,908cd9b8e8,49,F
3,dceb8ce861,46,U
4,f4f0ac6b06,119,U


## First issue date

In [9]:
data = data.merge(clients[['client_id', 'first_issue_date']], on='client_id', how='left')
data['first_issue_date'] = \
    (pd.to_datetime(data['first_issue_date'])- pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
data.head()

,client_id,age,gender,first_issue_date
0,a9a604ed6e,36,F,1536860201
1,ebd7360016,63,F,1499101932
2,908cd9b8e8,49,F,1522429280
3,dceb8ce861,46,U,1507309602
4,f4f0ac6b06,119,U,1530135581


## First redeem date

In [10]:
data = data.merge(clients[['client_id', 'first_redeem_date']], on='client_id', how='left')
data['first_redeem_date'] = \
    (pd.to_datetime(data['first_redeem_date'])- pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date
0,a9a604ed6e,36,F,1536860201,NaN
1,ebd7360016,63,F,1499101932,1.504283e+09
2,908cd9b8e8,49,F,1522429280,1.531502e+09
3,dceb8ce861,46,U,1507309602,1.534013e+09
4,f4f0ac6b06,119,U,1530135581,1.550262e+09


## Issue-redeem delay

In [11]:
data['issue_redeem_delay'] = data['first_redeem_date'] - data['first_issue_date']
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay
0,a9a604ed6e,36,F,1536860201,NaN,NaN
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0


## Number of days of card ownership

In [12]:
date_x = purchase['transaction_datetime'].apply(lambda x: date.fromisoformat(x.split()[0])).to_numpy().max()
df = clients.drop(columns=['first_redeem_date', 'age', 'gender'])
df['first_issue_date'] = df['first_issue_date'].apply(lambda d: date.fromisoformat(d.split()[0]))
df['card_own'] = df['first_issue_date'].apply(lambda d: (date_x - d).days)

data = data.merge(df[['client_id', 'card_own']], on='client_id', how='left')
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264


## Number of days after the first deduction of points

In [13]:
date_x = purchase['transaction_datetime'].apply(lambda x: date.fromisoformat(x.split()[0])).to_numpy().max()
df = clients.drop(columns=['first_issue_date', 'age', 'gender'])
df = df.fillna(method='bfill').fillna(method='ffill')
df['first_redeem_date'] = df['first_redeem_date'].apply(lambda d: date.fromisoformat(d.split()[0]))
df['from_redeem'] = df['first_redeem_date'].apply(lambda d: (date_x - d).days)

data = data.merge(df[['client_id', 'from_redeem']], on='client_id', how='left')
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31


## The client bought the own trademark products

In [14]:
df = purchase[['client_id', 'product_id']]
df = df.merge(products[['product_id', 'is_own_trademark']], on='product_id', how='left')
df = df.groupby(by=['client_id']).sum()

data['trademark'] = data['client_id'].apply(lambda client: int(df.loc[client] > 0))
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1


## The client bought a popular product

In [15]:
df = purchase[['client_id', 'product_id']]
df = df.groupby(by=['product_id']).count()
popular = df.idxmax().values[0]
popular

'4009f09b04'

In [16]:
def is_buy_popular(client):
    try:
        return (df.loc[client, 'product_id'] == popular).any()
    except:
        return df.loc[client, 'product_id'] == popular


df = purchase[['client_id', 'product_id']].set_index('client_id')
data['popular'] = data['client_id'].apply(is_buy_popular).astype(int)
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1


## Most of the customer's purchases have a receipt larger than the average receipt for the entire sample

In [17]:
df = purchase[['client_id', 'transaction_id', 'purchase_sum']].drop_duplicates('transaction_id')
df = df.set_index('client_id')
mean = df.purchase_sum.mean()
mean

427.9671216027777

In [18]:
data['receipt'] = data['client_id'].apply(lambda client: 
                                         (df.loc[client, 'purchase_sum'] > mean).sum() 
                                          > df.loc[client].shape[0] / 2).astype(int)
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,receipt
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,1
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,0
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,0
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,1
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,1


## The amount of regular points received

In [19]:
df = purchase[['client_id', 
               'transaction_id', 
               'regular_points_received']].drop_duplicates('transaction_id').set_index('client_id')

data['regularR'] = data['client_id'].apply(lambda client: df.loc[client, 'regular_points_received'].sum())
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,receipt,regularR
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,1,52.3
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,0,37.5
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,0,37.4
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,1,67.3
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,1,113.3


## The amount of regular points spent

In [20]:
df = purchase[['client_id', 
               'transaction_id', 
               'regular_points_spent']].drop_duplicates('transaction_id').set_index('client_id')

data['regularS'] = data['client_id'].apply(lambda client: df.loc[client, 'regular_points_spent'].sum())
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,receipt,regularR,regularS
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,1,52.3,0.0
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,0,37.5,-45.0
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,0,37.4,-79.0
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,1,67.3,-813.0
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,1,113.3,-114.0


## The amount of express points received

In [21]:
df = purchase[['client_id', 
               'transaction_id', 
               'express_points_received']].drop_duplicates('transaction_id').set_index('client_id')

data['expresR'] = data['client_id'].apply(lambda client: df.loc[client, 'express_points_received'].sum())
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,receipt,regularR,regularS,expresR
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,1,52.3,0.0,0.0
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,0,37.5,-45.0,0.0
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,0,37.4,-79.0,0.0
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,1,67.3,-813.0,40.0
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,1,113.3,-114.0,0.0


## The amount of express points spent

In [22]:
df = purchase[['client_id', 
               'transaction_id', 
               'express_points_spent']].drop_duplicates('transaction_id').set_index('client_id')

data['expresS'] = data['client_id'].apply(lambda client: df.loc[client, 'express_points_spent'].sum())
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,receipt,regularR,regularS,expresR,expresS
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,1,52.3,0.0,0.0,0.0
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,0,37.5,-45.0,0.0,0.0
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,0,37.4,-79.0,0.0,0.0
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,1,67.3,-813.0,40.0,0.0
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,1,113.3,-114.0,0.0,0.0


## Number of purchases

In [23]:
df = purchase[['client_id', 'transaction_id']].drop_duplicates('transaction_id').set_index('client_id')

data['n_purchases'] = data['client_id'].apply(lambda client: df.loc[client].shape[0])
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,receipt,regularR,regularS,expresR,expresS,n_purchases
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,1,52.3,0.0,0.0,0.0,5
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,0,37.5,-45.0,0.0,0.0,31
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,0,37.4,-79.0,0.0,0.0,17
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,1,67.3,-813.0,40.0,0.0,6
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,1,113.3,-114.0,0.0,0.0,12


## Sum of purchases

In [24]:
df = purchase[['client_id', 
               'transaction_id', 
               'purchase_sum']].drop_duplicates('transaction_id').drop(columns=['transaction_id'])
df = df.groupby(by=['client_id']).sum()

data['full_sum'] = data['client_id'].apply(lambda client: df.loc[client])
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,receipt,regularR,regularS,expresR,expresS,n_purchases,full_sum
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,1,52.3,0.0,0.0,0.0,5,3147.53
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,0,37.5,-45.0,0.0,0.0,31,5680.57
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,0,37.4,-79.0,0.0,0.0,17,6193.04
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,1,67.3,-813.0,40.0,0.0,6,8265.26
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,1,113.3,-114.0,0.0,0.0,12,8960.00


## Mean sum of purchases

In [25]:
df = purchase[['client_id', 
               'transaction_id', 
               'purchase_sum']].drop_duplicates('transaction_id').drop(columns=['transaction_id'])
df = df.groupby(by=['client_id']).mean()

data['mean_sum'] = data['client_id'].apply(lambda client: df.loc[client])
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,receipt,regularR,regularS,expresR,expresS,n_purchases,full_sum,mean_sum
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,1,52.3,0.0,0.0,0.0,5,3147.53,629.506000
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,0,37.5,-45.0,0.0,0.0,31,5680.57,183.244194
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,0,37.4,-79.0,0.0,0.0,17,6193.04,364.296471
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,1,67.3,-813.0,40.0,0.0,6,8265.26,1377.543333
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,1,113.3,-114.0,0.0,0.0,12,8960.00,746.666667


## Max sum of purchases

In [26]:
df = purchase[['client_id', 
               'transaction_id', 
               'purchase_sum']].drop_duplicates('transaction_id').drop(columns=['transaction_id'])
df = df.groupby(by=['client_id']).max()

data['max_sum'] = data['client_id'].apply(lambda client: df.loc[client])
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,receipt,regularR,regularS,expresR,expresS,n_purchases,full_sum,mean_sum,max_sum
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,1,52.3,0.0,0.0,0.0,5,3147.53,629.506000,971.67
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,0,37.5,-45.0,0.0,0.0,31,5680.57,183.244194,407.00
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,0,37.4,-79.0,0.0,0.0,17,6193.04,364.296471,689.14
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,1,67.3,-813.0,40.0,0.0,6,8265.26,1377.543333,2568.53
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,1,113.3,-114.0,0.0,0.0,12,8960.00,746.666667,1952.00


## Min sum of purchases

In [27]:
df = purchase[['client_id', 
               'transaction_id', 
               'purchase_sum']].drop_duplicates('transaction_id').drop(columns=['transaction_id'])
df = df.groupby(by=['client_id']).min()

data['min_sum'] = data['client_id'].apply(lambda client: df.loc[client])
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,receipt,regularR,regularS,expresR,expresS,n_purchases,full_sum,mean_sum,max_sum,min_sum
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,1,52.3,0.0,0.0,0.0,5,3147.53,629.506000,971.67,99.0
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,0,37.5,-45.0,0.0,0.0,31,5680.57,183.244194,407.00,47.0
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,0,37.4,-79.0,0.0,0.0,17,6193.04,364.296471,689.14,59.0
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,1,67.3,-813.0,40.0,0.0,6,8265.26,1377.543333,2568.53,473.0
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,1,113.3,-114.0,0.0,0.0,12,8960.00,746.666667,1952.00,381.0


## Client bought alcohol

In [28]:
df = purchase[['client_id', 'product_id']]
df = df.merge(products[['product_id', 'is_alcohol']], on='product_id', how='left')
df = df.groupby(by=['client_id']).sum()

data['alcohol'] = data['client_id'].apply(lambda client: int(df.loc[client] > 0))
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,...,regularR,regularS,expresR,expresS,n_purchases,full_sum,mean_sum,max_sum,min_sum,alcohol
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,...,52.3,0.0,0.0,0.0,5,3147.53,629.506000,971.67,99.0,0
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,...,37.5,-45.0,0.0,0.0,31,5680.57,183.244194,407.00,47.0,0
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,...,37.4,-79.0,0.0,0.0,17,6193.04,364.296471,689.14,59.0,0
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,...,67.3,-813.0,40.0,0.0,6,8265.26,1377.543333,2568.53,473.0,0
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,...,113.3,-114.0,0.0,0.0,12,8960.00,746.666667,1952.00,381.0,1


## Average mass of client purchases

In [29]:
df = purchase[['client_id', 'transaction_id', 'product_id']]
df = df.merge(products[['product_id', 'netto']], on='product_id', how='left').drop(columns=['product_id'])
df = df.groupby(by=['client_id', 'transaction_id']).sum()
df = df.groupby(by=['client_id']).mean()

data['netto'] = data['client_id'].apply(lambda client: df.loc[client])
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,...,regularS,expresR,expresS,n_purchases,full_sum,mean_sum,max_sum,min_sum,alcohol,netto
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,...,0.0,0.0,0.0,5,3147.53,629.506000,971.67,99.0,0,3.827000
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,...,-45.0,0.0,0.0,31,5680.57,183.244194,407.00,47.0,0,1.511548
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,...,-79.0,0.0,0.0,17,6193.04,364.296471,689.14,59.0,0,2.527588
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,...,-813.0,40.0,0.0,6,8265.26,1377.543333,2568.53,473.0,0,8.174500
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,...,-114.0,0.0,0.0,12,8960.00,746.666667,1952.00,381.0,1,5.209417


## Average frequency of purchases

In [30]:
date_x = purchase['transaction_datetime'].apply(lambda x: date.fromisoformat(x.split()[0])).to_numpy().max()
df = purchase[['client_id', 
               'transaction_id', 
               'transaction_datetime']].drop_duplicates('transaction_id').set_index('client_id')
df['transaction_datetime'] = df['transaction_datetime'].apply(lambda x: date.fromisoformat(x.split()[0]))
df = df.drop(columns=['transaction_id'])

data['frequency'] = data['client_id'].apply(lambda client: 
                                            np.diff(np.append(df.loc[client].to_numpy().reshape(-1), date_x))
                                            .mean().days)
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,...,expresR,expresS,n_purchases,full_sum,mean_sum,max_sum,min_sum,alcohol,netto,frequency
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,...,0.0,0.0,5,3147.53,629.506000,971.67,99.0,0,3.827000,19
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,...,0.0,0.0,31,5680.57,183.244194,407.00,47.0,0,1.511548,3
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,...,0.0,0.0,17,6193.04,364.296471,689.14,59.0,0,2.527588,6
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,...,40.0,0.0,6,8265.26,1377.543333,2568.53,473.0,0,8.174500,11
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,...,0.0,0.0,12,8960.00,746.666667,1952.00,381.0,1,5.209417,9


## Time since last purchase

In [31]:
date_x = purchase['transaction_datetime'].apply(lambda x: date.fromisoformat(x.split()[0])).to_numpy().max()

df = purchase[['client_id', 
               'transaction_id', 
               'transaction_datetime']].drop_duplicates('transaction_id').set_index('client_id')
df['transaction_datetime'] = df['transaction_datetime'].apply(lambda x: date.fromisoformat(x.split()[0]))
df = df.drop(columns=['transaction_id'])
df = df.groupby(by=['client_id']).max()

data['last_p'] = data['client_id'].apply(lambda client: (date_x - df.loc[client, 'transaction_datetime']).days)
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,...,expresS,n_purchases,full_sum,mean_sum,max_sum,min_sum,alcohol,netto,frequency,last_p
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,...,0.0,5,3147.53,629.506000,971.67,99.0,0,3.827000,19,10
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,...,0.0,31,5680.57,183.244194,407.00,47.0,0,1.511548,3,1
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,...,0.0,17,6193.04,364.296471,689.14,59.0,0,2.527588,6,12
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,...,0.0,6,8265.26,1377.543333,2568.53,473.0,0,8.174500,11,13
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,...,0.0,12,8960.00,746.666667,1952.00,381.0,1,5.209417,9,13


## The client bought in a popular store

In [32]:
df = purchase[['store_id', 'transaction_id']].drop_duplicates('transaction_id')
df = df.groupby(by=['store_id']).count()
popular = df.idxmax().values[0]
popular

'1f41964607'

In [33]:
def is_buy_popular(client):
    try:
        return (df.loc[client, 'store_id'] == popular).any()
    except:
        return df.loc[client, 'store_id'] == popular


df = purchase[['client_id', 'store_id']].set_index('client_id')
data['popular_store'] = data['client_id'].apply(is_buy_popular).astype(int)
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,...,n_purchases,full_sum,mean_sum,max_sum,min_sum,alcohol,netto,frequency,last_p,popular_store
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,...,5,3147.53,629.506000,971.67,99.0,0,3.827000,19,10,0
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,...,31,5680.57,183.244194,407.00,47.0,0,1.511548,3,1,0
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,...,17,6193.04,364.296471,689.14,59.0,0,2.527588,6,12,0
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,...,6,8265.26,1377.543333,2568.53,473.0,0,8.174500,11,13,0
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,...,12,8960.00,746.666667,1952.00,381.0,1,5.209417,9,13,0


## The client bought products from popular brand

In [34]:
df = purchase[['product_id']].copy()
df = df.merge(products[['product_id', 'brand_id']], on='product_id', how='left')
df = df.groupby(by=['brand_id']).count()
popular = df.idxmax().values[0]
popular

'4da2dc345f'

In [35]:
def is_buy_popular(client):
    try:
        return (df.loc[client, 'brand_id'] == popular).any()
    except:
        return df.loc[client, 'brand_id'] == popular


df = purchase[['client_id', 'product_id']].copy()
df = df.merge(products[['product_id', 'brand_id']], on='product_id', how='left').set_index('client_id')
data['popular_brand'] = data['client_id'].apply(is_buy_popular).astype(int)
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,...,full_sum,mean_sum,max_sum,min_sum,alcohol,netto,frequency,last_p,popular_store,popular_brand
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,...,3147.53,629.506000,971.67,99.0,0,3.827000,19,10,0,1
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,...,5680.57,183.244194,407.00,47.0,0,1.511548,3,1,0,1
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,...,6193.04,364.296471,689.14,59.0,0,2.527588,6,12,0,1
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,...,8265.26,1377.543333,2568.53,473.0,0,8.174500,11,13,0,1
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,...,8960.00,746.666667,1952.00,381.0,1,5.209417,9,13,0,1


## The client bought products from popular category in lvl3

In [36]:
df = purchase[['product_id']].copy()
df = df.merge(products[['product_id', 'level_3']], on='product_id', how='left')
df = df.groupby(by=['level_3']).count()
popular = df.idxmax().values[0]
popular

'ca69ed9de2'

In [37]:
def is_buy_popular(client):
    try:
        return (df.loc[client, 'level_3'] == popular).any()
    except:
        return df.loc[client, 'level_3'] == popular


df = purchase[['client_id', 'product_id']].copy()
df = df.merge(products[['product_id', 'level_3']], on='product_id', how='left').set_index('client_id')
data['popular_lvl3'] = data['client_id'].apply(is_buy_popular).astype(int)
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,...,mean_sum,max_sum,min_sum,alcohol,netto,frequency,last_p,popular_store,popular_brand,popular_lvl3
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,...,629.506000,971.67,99.0,0,3.827000,19,10,0,1,1
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,...,183.244194,407.00,47.0,0,1.511548,3,1,0,1,1
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,...,364.296471,689.14,59.0,0,2.527588,6,12,0,1,1
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,...,1377.543333,2568.53,473.0,0,8.174500,11,13,0,1,1
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,...,746.666667,1952.00,381.0,1,5.209417,9,13,0,1,1


##  The client bought products from popular vendor

In [38]:
df = purchase[['product_id']].copy()
df = df.merge(products[['product_id', 'vendor_id']], on='product_id', how='left')
df = df.groupby(by=['vendor_id']).count()
popular = df.idxmax().values[0]
popular

'43acd80c1a'

In [39]:
def is_buy_popular(client):
    try:
        return (df.loc[client, 'vendor_id'] == popular).any()
    except:
        return df.loc[client, 'vendor_id'] == popular


df = purchase[['client_id', 'product_id']].copy()
df = df.merge(products[['product_id', 'vendor_id']], on='product_id', how='left').set_index('client_id')
data['popular_vr'] = data['client_id'].apply(is_buy_popular).astype(int)
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,...,max_sum,min_sum,alcohol,netto,frequency,last_p,popular_store,popular_brand,popular_lvl3,popular_vr
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,...,971.67,99.0,0,3.827000,19,10,0,1,1,1
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,...,407.00,47.0,0,1.511548,3,1,0,1,1,1
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,...,689.14,59.0,0,2.527588,6,12,0,1,1,1
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,...,2568.53,473.0,0,8.174500,11,13,0,1,1,1
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,...,1952.00,381.0,1,5.209417,9,13,0,1,1,1


## Sum of return redeem

In [40]:
df = purchase[['client_id', 'trn_sum_from_red']].copy()
df = df.groupby(by=['client_id']).sum()

data['trn_red'] = data['client_id'].apply(lambda client: df.loc[client, 'trn_sum_from_red'])
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,...,min_sum,alcohol,netto,frequency,last_p,popular_store,popular_brand,popular_lvl3,popular_vr,trn_red
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,...,99.0,0,3.827000,19,10,0,1,1,1,0.0
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,...,47.0,0,1.511548,3,1,0,1,1,1,60.0
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,...,59.0,0,2.527588,6,12,0,1,1,1,282.0
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,...,473.0,0,8.174500,11,13,0,1,1,1,1093.0
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,...,381.0,1,5.209417,9,13,0,1,1,1,2025.0


## Sum of return issue

In [41]:
df = purchase[['client_id', 'trn_sum_from_iss']].copy()
df = df.groupby(by=['client_id']).sum()

data['trn_iss'] = data['client_id'].apply(lambda client: df.loc[client, 'trn_sum_from_iss'])
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,...,alcohol,netto,frequency,last_p,popular_store,popular_brand,popular_lvl3,popular_vr,trn_red,trn_iss
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,...,0,3.827000,19,10,0,1,1,1,0.0,3045.0
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,...,0,1.511548,3,1,0,1,1,1,60.0,5659.0
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,...,0,2.527588,6,12,0,1,1,1,282.0,6116.0
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,...,0,8.174500,11,13,0,1,1,1,1093.0,7452.0
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,...,1,5.209417,9,13,0,1,1,1,2025.0,8743.0


## Number of purchases of a popular product

In [42]:
df = purchase[['client_id', 'product_id']]
df = df.groupby(by=['product_id']).count()
popular = df.idxmax().values[0]
popular

'4009f09b04'

In [43]:
def is_buy_popular(client):
    try:
        return (df.loc[client, 'product_id'] == popular).sum()
    except:
        return df.loc[client, 'product_id'] == popular


df = purchase[['client_id', 'product_id']].set_index('client_id')
data['popular_cnt'] = data['client_id'].apply(is_buy_popular).astype(int)
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,...,netto,frequency,last_p,popular_store,popular_brand,popular_lvl3,popular_vr,trn_red,trn_iss,popular_cnt
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,...,3.827000,19,10,0,1,1,1,0.0,3045.0,2
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,...,1.511548,3,1,0,1,1,1,60.0,5659.0,0
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,...,2.527588,6,12,0,1,1,1,282.0,6116.0,2
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,...,8.174500,11,13,0,1,1,1,1093.0,7452.0,0
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,...,5.209417,9,13,0,1,1,1,2025.0,8743.0,8


## The number of checks is above average

In [44]:
df = purchase[['client_id', 'transaction_id', 'purchase_sum']].drop_duplicates('transaction_id')
df = df.set_index('client_id')
mean = df.purchase_sum.mean()
mean

427.9671216027777

In [45]:
data['receipt_cnt'] = data['client_id'].apply(lambda client: (df.loc[client, 'purchase_sum'] > mean).sum())
data.head()

,client_id,age,gender,first_issue_date,first_redeem_date,issue_redeem_delay,card_own,from_redeem,trademark,popular,...,frequency,last_p,popular_store,popular_brand,popular_lvl3,popular_vr,trn_red,trn_iss,popular_cnt,receipt_cnt
0,a9a604ed6e,36,F,1536860201,NaN,NaN,186,642,1,1,...,19,10,0,1,1,1,0.0,3045.0,2,4
1,ebd7360016,63,F,1499101932,1.504283e+09,5180580.0,623,563,1,0,...,3,1,0,1,1,1,60.0,5659.0,0,0
2,908cd9b8e8,49,F,1522429280,1.531502e+09,9072272.0,353,248,1,1,...,6,12,0,1,1,1,282.0,6116.0,2,7
3,dceb8ce861,46,U,1507309602,1.534013e+09,26703436.0,528,219,1,0,...,11,13,0,1,1,1,1093.0,7452.0,0,6
4,f4f0ac6b06,119,U,1530135581,1.550262e+09,20126096.0,264,31,1,1,...,9,13,0,1,1,1,2025.0,8743.0,8,9


# Save

In [46]:
data.to_csv('artifacts/data_' + MODE + '.csv', index=False)